<a href="https://colab.research.google.com/github/aarongraybill/Math210/blob/main/ProblemSets/Midterm1/Midterm1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Math 210
## Aaron Graybill
### Midterm 1
### 3/18/21

## Problem 1.
First let me bring in all of the predefined scripts and packages:

In [2]:
import numpy as np

In [3]:
def print_tableau(a,indep_names,dep_names):
#
# Given matrix "a" and lists of variables names "indep_names" and "dep_names",
# this function prints the matrix and labels in standard tableau format
# (including adding the -1, the minus signs in the last column, and labeling the lower-right as obj)
#
# First, check the inputs: indep_names should be one shorter than the number of columns of A
#                          dep_names should be one shorter than the number of rows of A
#
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in A
    ncols = a.shape[1]
    nindep = len(indep_names)
    ndep = len(dep_names)
    if nindep != ncols-1:
        print("WARNING: # of indep vbles should be one fewer than # columns of matrix")
    if ndep != nrows-1:
        print("WARNING: # of dep vbles should be one fewer than # rows of matrix")
# Now do the printing (uses a variety of formatting techniques in Python)        
    for j in range(ncols-1):                    # Print the independent variables in the first row
        print(indep_names[j].rjust(10),end="")  # rjust(10) makes fields 10 wide and right-justifies;
                                                #    the end command prevents newline)
    print("        -1")                         # Tack on the -1 at the end of the first row
    for i in range(nrows-1):
        for j in range(ncols):                  # Print all but the last row of the matrix
            print("%10.3f" % a[i][j],end="") # The syntax prints in a field 10 wide, showing 3 decimal points
        lab = "= -" + dep_names[i]
        print(lab.rjust(10))
    for j in range(ncols):
        print("%10.3f" % a[nrows-1][j],end="")  # Print the last row of the matrix, with label "obj" at end
    lab = "= obj"
    print(lab.rjust(10))
    print(" ")    # Put blank line at bottom

In [4]:
def pivot(a,pivrow,pivcol,indep_names,dep_names) :
# 
# Given matrix "a", a row number "pivrow" and column number "pivcol", 
#  and lists of variable names "indep_names" and "dep_names", this
#  function does three things:
#    (1) outputs the new version of the matrix after a pivot,
#    (2) updates the lists of variable names post-pivot
#    (3) prints the new matrix, including labels showing the variable names
#
# First, check the inputs: indep_names should be one shorter than the number of columns of A
#                          dep_names should be one shorter than the number of rows of A
#                          you should not be pivoting on the last row or last column
#
    a = a.astype(float)   # make sure entries are treated as floating point numbers
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in A
    ncols = a.shape[1]
    nindep = len(indep_names)
    ndep = len(dep_names)
    if nindep != ncols-1:
        print("WARNING: # of indep vbles should be one fewer than # columns of matrix")
    if ndep != nrows-1:
        print("WARNING: # of dep vbles should be one fewer than # rows of matrix")
    if pivrow > nrows-1 or pivcol > ncols-1:
        print("WARNING: should not pivot on last row or column")
    newa = a.copy()       # make a copy of A, to be filled in below with result of pivot
    p = a[pivrow-1][pivcol-1]   # identify pivot element
    newa[pivrow-1][pivcol-1] = 1/p   # set new value of pivot element
    # Set entries in p's row
    for j in range(ncols):
        if j != pivcol-1:
            newa[pivrow-1][j]=a[pivrow-1][j]/p;
    # Set entries in p's column
    for i in range(nrows):
        if i != pivrow-1:
            newa[i][pivcol-1]=-a[i][pivcol-1]/p;
    # Set all other entries
    for i in range(nrows):
        for j in range(ncols):
            if i != pivrow-1 and j != pivcol-1:
                r = a[i][pivcol-1]
                q = a[pivrow-1][j]
                s = a[i][j]
                newa[i][j]=(p*s-q*r)/p
    # Now swap the variable names
    temp = indep_names[pivcol-1]
    indep_names[pivcol-1]=dep_names[pivrow-1]
    dep_names[pivrow-1]=temp
    print_tableau(newa,indep_names,dep_names) # Print the matrix with updated labels
    return newa;

In [5]:
def target(a) :
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in "a"
    ncols = a.shape[1]
    import numpy as np
    v = np.empty(ncols-1)
    for i in range(ncols-1):
        v[i]=a[nrows-1,i]
    biggest_c = np.max(v)
    where_is_biggest_c = np.argmax(v)+1
    if biggest_c > 0 :
        return where_is_biggest_c
    else :
        return -1

In [36]:
def select(a,pivcolnum) :
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in A
    ncols = a.shape[1]
# First task: work down the column and record the b/a ratios in a vector v
#     except record -1 if a is negative or zero
    import numpy as np
    v = np.zeros(nrows-1)
    for i in range(nrows-1):
        if a[i,pivcolnum-1]>0 :
            v[i] = a[i,ncols-1]/a[i,pivcolnum-1]
        else :
            v[i] = -1
# Second task: if max b/a > -1, find min b/a by hand (ignoring zero entries in v)
    if np.max(v) > -1 :  
        min_so_far = np.max(v)+1  # Initialize variable to be for-sure bigger than the min
        for i in range(nrows-1):
            if v[i] > -1 and v[i] < min_so_far :
                min_so_far = v[i]
                where_is_min = i+1   # Add 1 to use human numbering
        return where_is_min      # Once we've scanned v for min, we can return result
    else :          
        return -1

In [7]:
# Create Simplex BF
def SimplexBF(a,indep_names,dep_names):
  nrows, ncols = a.shape
  a_new = a
  print_tableau(a_new,indep_names,dep_names)
  while np.max(a_new[nrows-1,:-1])>0:
    pivcol=target(a_new)
    pivrow=select(a_new,pivcol)
    if pivrow == -1:
      return("Unbounded")
    else:
      a_new=pivot(a_new,pivrow,pivcol,indep_names,dep_names)
      print_tableau(a_new,indep_names,dep_names)



### a.
Okay now I can start on SimplexNBF:

In [8]:
def targetnbf(tab):
  nrows, ncols = tab.shape
  new_i = -1
  for i in range(nrows-1) :# don't check obj fn row
    if tab[i,ncols-1] < 0:
      new_i = i+1
  return(new_i)

### b.

In [9]:
def candidateone(tab,targetedrow):
  #don't specify a row that's the obj fn row, I have no protocol against it
  nrows, ncols = tab.shape
  for i in range(ncols-1): #don't check last col bc it's the b's
    if tab[targetedrow-1,i]<0 :
      return(i+1)#bailout if found one
  return(-1) #none found in loop, return -1


### c.

In [10]:
def  selectnbf(tab,targetedrow,pivcolumn) :
  nrows, ncols = tab.shape

  #subset of two columns in question exclude obj fn row
  candidate_column =tab[targetedrow-1:nrows-1,pivcolumn-1]
  b_column=tab[targetedrow-1:nrows-1,ncols-1]

  # compute ratios
  ratios=b_column/candidate_column

  # Find row in subset and convert back to index in full table:
  #don't need to start at first row, let those be the starting values
  cur_ratio= ratios[0]
  best_row=0
  # the seemingly misplaced -1s and +1s are because we start at second row
  for i in range(len(candidate_column)-1):
    if candidate_column[i+1]>0 and ratios[i+1]<cur_ratio :
      cur_ratio=ratios[i+1]
      best_row=i+1

  #the targetedrow already has the +1, so this converts to start at 1
    #and converts table subset index to full tableau index
  best_row=best_row+targetedrow  

  return([best_row,pivcolumn]) #output

### d.

In [21]:
def simplexnbf(tab,indep_names,dep_names):
  nrows, ncols = tab.shape #get dims
  current_target_row=np.Inf #set current target row to nonsense
  tab_new=tab # get primer value for tableau iterator
  
  NotReadyForBF = True #changed when problem is BF
  SolutionPossible = True #false when no solution is discovered

  print_tableau(tab,indep_names,dep_names) #print initial tableau

  # while NBF and solution still possible apply NBF rules
  while NotReadyForBF and SolutionPossible:
    current_target_row=targetnbf(tab_new) # find target
    if current_target_row ==-1 : #if target is -1, Ready for BF
      NotReadyForBF = False
    else : #find candidate from targeted row
      current_candidate_column=candidateone(tab_new,current_target_row)
      if current_candidate_column ==-1 : #if no candidates, no solution
        SolutionPossible = False
      else : #pivot based off of the computed selection
        pivot_row,pivot_col=selectnbf(tab_new,current_target_row,current_candidate_column)
        tab_new=pivot(tab_new,pivot_row,pivot_col,indep_names,dep_names)
  if not NotReadyForBF : #if we bailed because ready for BF apply SimplexBF
    SimplexBF(tab_new,indep_names,dep_names)
  if not SolutionPossible: # if we bailed bc constraint set emtpy, say so 
    return("-1, Constraint set empty, sorry ☹️")
  
  #we can't exit the while loop for anything but the reasons above, 
  # so don't need any else statements
  


That's actually not what the question is asking for, so let me tighten it up to give the requested output (though you can gather the desired result from the above).

In [12]:
def simplexnbf_final(tab,indep_names,dep_names):
  output = simplexnbf(tab,indep_names,dep_names)
  # my sad error message is a string, so when we see that, return -1
  if output=="-1, Constraint set empty, sorry ☹️": 
    return -1
  #else we must have a tableau for BF, return 0
  else : 
    return 0

Here is proof that all of my functions work as desired.

In [32]:
a=np.array([
            [-2,-10,3,-20,8],
            [-1,1,0,-3,-6],
            [1,4,-1,8,-1],
            [1,1,1,1,0]])
indep_names=["x","y","z","w"]
dep_names=["t1","t2","t3"]

print(targetnbf(a))
print(candidateone(a,3))
print(selectnbf(a,3,1))


simplexnbf(a,indep_names,dep_names)

3
3
[3, 1]
         x         y         z         w        -1
    -2.000   -10.000     3.000   -20.000     8.000     = -t1
    -1.000     1.000     0.000    -3.000    -6.000     = -t2
     1.000     4.000    -1.000     8.000    -1.000     = -t3
     1.000     1.000     1.000     1.000     0.000     = obj
 
         x         y        t3         w        -1
     1.000     2.000     3.000     4.000     5.000     = -t1
    -1.000     1.000     0.000    -3.000    -6.000     = -t2
    -1.000    -4.000    -1.000    -8.000     1.000      = -z
     2.000     5.000     1.000     9.000    -1.000     = obj
 
        t2         y        t3         w        -1
     1.000     3.000     3.000     1.000    -1.000     = -t1
    -1.000    -1.000    -0.000     3.000     6.000      = -x
    -1.000    -5.000    -1.000    -5.000     7.000      = -z
     2.000     7.000     1.000     3.000   -13.000     = obj
 


'-1, Constraint set empty, sorry ☹️'

In [34]:
indep_names=["x","y","z","w"]
dep_names=["t1","t2","t3"]

simplexnbf_final(a,indep_names,dep_names)

         x         y         z         w        -1
    -2.000   -10.000     3.000   -20.000     8.000     = -t1
    -1.000     1.000     0.000    -3.000    -6.000     = -t2
     1.000     4.000    -1.000     8.000    -1.000     = -t3
     1.000     1.000     1.000     1.000     0.000     = obj
 
         x         y        t3         w        -1
     1.000     2.000     3.000     4.000     5.000     = -t1
    -1.000     1.000     0.000    -3.000    -6.000     = -t2
    -1.000    -4.000    -1.000    -8.000     1.000      = -z
     2.000     5.000     1.000     9.000    -1.000     = obj
 
        t2         y        t3         w        -1
     1.000     3.000     3.000     1.000    -1.000     = -t1
    -1.000    -1.000    -0.000     3.000     6.000      = -x
    -1.000    -5.000    -1.000    -5.000     7.000      = -z
     2.000     7.000     1.000     3.000   -13.000     = obj
 


-1

## Problem 2

### a.
Since we pivot on $a_{62}$ using the pivot formula we have the following values for the new $b_i$s:

$$
b_i^{new}=
\begin{cases}
\frac{b_i}{a_{62}} & i =6\\
\frac{a_{62}b_i-a_{i2}b_6}{a_{62}} & i\neq6
\end{cases}
$$
Plugging in the zeros and simplyfing gives:

$$
b_i^{new}=
\begin{cases}
0 & i =6\\
b_i & i\neq6
\end{cases}
$$

But since $b_6=0$, we are back where we started with all of the $b$s reamining the same. 

### b.
Let $a_{ij}^n$ be the value $a_{ij}$ in tableau $T_n$.

Proof by contradiction, suppose that $a_{62}^2$ was indeed the pivot for $T^2$. Since all of the $b_i^2$s remain the same in $T_2$ as the $b_i^1$s in $T_1$. we must again target row four.

To pivot on $a_{62}^2$, by the candidate rules, $a_{42}^2<0$.

Since we pivoted on $a_{62}^1$, $a_{62}^1>0$ (bc $b_6>0$ or it would be target). For $T_1$, we targeted row 4, so $a_{42}^1<0$ again by the candidate rules.

Using the pivot rules, I compute $a_{42}^2$ as follows:
$$
a_{42}^2=-\frac{a_{42}^1}{a_{62}^1}
$$
From the results above the numerator must be negative while the denominator must be positive, so the whole fraction when multiplied by $-1$ is positive. Therefore $a_{42}^2>0$. Which contradicts the earlier result. Therefore the supposition that $a_{62}^2$ is the pivot must be incorrect. We pivot elsewhere.  $\blacksquare$

## Problem 3.

### a.
There are two ways to have no minimum. First if the constraint set is empty which in this case it is not (see plot below). Or if the objective function is unconstrained in a direction that decreases the objective function to decrease ad infiinitum. For the function objective function $y-Ax$ to be decreasing while $x,y\geq0$, at least one of those terms has to be negative. Therefore $A>0$. 

Now if the max exists, it must occur at a finite point. By the geometric method, we know that this point must be at the intersection of wto constraints (because there are two indep vars). The plot below outlines which intersections form the corners of the region.


In [47]:
%%HTML
<iframe src="https://www.desmos.com/calculator/6y1stgwg3x" width="1000px" height="500px" style="border: 1px solid #ccc" frameborder=0></iframe>

Reading off of the graph above, we have that the corners at which a maximum could occur are at the intersection of the edges of the 1st and 3rd constraints and the 2nd and 3rd constraints.

Solving these systems gives that the maximum must occcur at either $\left(\frac{28}{5},\frac{92}{5}\right)$ or $\left(\frac{69}{4},\frac{27}{4}\right)$

## Problem 4.
Thankfully, the problem is already in the proper form to be added to the tableau, though we will have to run NBF.
Note that the objective function is asking us to find the largest value of $x_4$ which means all of the other $x_i$s add nothing and each $x-4$ is worth one. The objective function must then be $f(X)=x_4$
I setup the tableau by reading off of the constraints and applying SimplexNBF: 

In [51]:
a=np.array([[1,2,1,1,16],[-1,-1,2,3,5],[-1,2,-4,5,-14],[1,-4,-3,6,24],[0,0,0,1,0]])
indep_names=["x1","x2","x3","x4"]
dep_names=["t1","t2","t3","t4"]
simplexnbf(a,indep_names,dep_names)

        x1        x2        x3        x4        -1
     1.000     2.000     1.000     1.000    16.000     = -t1
    -1.000    -1.000     2.000     3.000     5.000     = -t2
    -1.000     2.000    -4.000     5.000   -14.000     = -t3
     1.000    -4.000    -3.000     6.000    24.000     = -t4
     0.000     0.000     0.000     1.000     0.000     = obj
 
        t3        x2        x3        x4        -1
     1.000     4.000    -3.000     6.000     2.000     = -t1
    -1.000    -3.000     6.000    -2.000    19.000     = -t2
    -1.000    -2.000     4.000    -5.000    14.000     = -x1
     1.000    -2.000    -7.000    11.000    10.000     = -t4
     0.000     0.000    -0.000     1.000    -0.000     = obj
 
        t3        x2        x3        x4        -1
     1.000     4.000    -3.000     6.000     2.000     = -t1
    -1.000    -3.000     6.000    -2.000    19.000     = -t2
    -1.000    -2.000     4.000    -5.000    14.000     = -x1
     1.000    -2.000    -7.000    11.000    10.000

I will briefly talk through those pivots because the question seems to be asking us to figure out the pivots by hand. First we have to run NBF because one of the $b_i$s is negative. As such we target row 3, which makes us take row one as the candidates and then we compute smallest ratio which happens to be in $a_{31}$. Pivoting there gives the next tableau in the sequence, which happens to be BF. 

We target the column with the highest value, that happens to be column 4. I compute the ratios that have positive denominators, and we select the smallest which happens to be $a_{14}$. Continue in a similar way until we get the following result:
\begin{align}
x_1^*&=9.767 \\
x_2^*&=0 \\
x_3^*&=3.933 \\
x_4^*&=2.3 \\
\end{align}
And the value of the objective function is 2.3, implying that the maximum value that $x_4$ can attain in the set is $2.3$ 

## Problem 5.
We have five choice variables, $\mathcal{A}_1$ and $\mathcal{A}_2$ the number of apartments built in periods one and two, $\mathcal{O}_1$ and $\mathcal{O}_2$ for the number of of offices built in period one and two and  $\mathcal{T}$ for the number of electricians sent to training in period one
The master electricians in period one is a static number and they can be allocated to either $\mathcal{A}$ or $\mathcal{O}$. Their constraint would be:
$$
\mathcal{A}_1+\mathcal{O}_1\leq 20\\
$$

In period two, the constraint is eased by the number of electricians sent to training. Therefore the $t=2$ master electrician constraint is 
$$
\mathcal{A}_2+\mathcal{O}_2\leq 20+\mathcal{T}\\
$$.

We have a similar constraint but going in the opposite direction for regular electricians. We send $\mathcal{T}$ away in the first period meaning:
$$
2\mathcal{A}_1+5\mathcal{O}_1 \leq120-\mathcal{T}
$$
Those electricians become masters in period two, so are also removed from that constraint giving the same constraint for $t=2$
$$
2\mathcal{A}_2+5\mathcal{O}_2 \leq120-\mathcal{T}
$$
We then have the carptenter and painter constraints which are more straightforward and give:
\begin{align*}
x
\begin{align*}